# Building a FloPy Plugin using the Stress Template and API Package

The MODFLOW-6 API supports a BMI interface which allows you to modify the behavior of MODFLOW-6.  FloPy plugins streamline the process of using MODFLOW-6's BMI interface by:

* Providing template generators to help rapidly set up the interface for your plugin
* Integrating FloPy plugins as a part of MODFLOW-6 simulations so that adding a FloPy plugin to your simulation is as easy as adding a MODFLOW-6 package to your simulation 
* Providing an interface that allows FloPy plugins to work together

In this notebook we will create a simple MODFLOW-6 plugin that behaves similarly to the MODFLOW-6 WEL package.  This plugin will be created using the FloPy template generator, which creates the FloPy plugin files you will need to get started quickly.  In this specific example the generic API package will be modified by this plugin to behave like a simplified WEL package.

To create this plugin, first define the model type and pick a three letter abbreviation for your plugin.

In [ ]:
# define the model type
mt = "gwf"

# define the stress plugin three letter abbreviation
pn = "npc"

Next define any user-specified settings for your plugin.  These settings will exist in a "package" file similar to the files used to store MODFLOW-6 package user-specified settings.  The FloPy plugin template generator can set up an options, package, and stress period block for your "package" settings file.

You can tell the template generator which options and settings to define by creating a dictionary where each key is an option/setting name and each value is a dictionary of attributes for that option/setting.  The only required attribute is "type".  Optional attributes include "shape" and "optional".

In [ ]:
# define the plugin options
opt = {"print_input": {"type": "keyword", "optional": "True"},
       "print_flows": {"type": "keyword", "optional": "True"},
       "save_input": {"type": "keyword", "optional": "True"},
       "save_flows": {"type": "keyword", "optional": "True"},
       "auxiliary": {"type": "string", "shape": "naux", "optional": "True"},
       "boundnames": {"type": "keyword", "optional": "True"}}

Define stress period data for your plugin in the same way.  You can also optionally define package data. 

In [ ]:
# define the stress plugin stress period data
spv = {"cellid": {"type": "integer", "shape": "ncelldim"},
       "q": {"type": "double precision"},
       "aux": {"type": "double precision", "shape": "naux",
               "optional": "True"},
       "boundname": {"type": "string", "optional": "True"}}

Now that all of your plugin interface information is defined, run the FloPy plugin template generator "generate_plugin_template", passing the information you just defined.

In [ ]:
import flopy

flopy.mf6.utils.flopy_plugins.plugin_template.generate_plugin_template(
    mt,
    pn,
    options=opt,
    stress_period_vars=spv)

## Working with flopy_npc_plugin.py

The plugin template generator creates and modifies several files.  The plugin code file that you will edit in order to add functionality to your plugin is in the in the python working folder (folder containing this notebook) and in this case is named "flopy_npc_plugin.py".  This file contains a class derived from FPBMIPluginInterface with several methods.  

### init_plugin() 
This method is called immediately before to the start of a model run.  The init_plugin method has been filled out for you to retreive all of your plugin settings from the plugin input file here.

### stress_period_start(self, sp, sln_group)
This method is called immediately after the MODFLOW-6 prepare_solve method is called for a new stress period.  The stress_period_start method has also been filled out for you, and the appropriate stress period data is loaded here.

This method receives the current stress period (sp) and the modflowapi interface to the solution group currently being solved for (sln_group).  For simulations with more than one solution group, the flopy plug-in is associated with a specific solution group based on what model it is a part of and the flopy plug-in callback occurrs only when that solution group is being solved.  To override this behavior and receiving a callback when all solution groups are beings solved, set the run_for_all_solution_groups attribute to True.

### time_step_start(self, sp, ts, sln_group)
This method is called immediately after MODFLOW-6's prepare_solve method is called.  

This method receives the current stress period (sp), the current time step (ts), and the modflowapi interface to the solution group currently being solved for (sln_group).  For simulations with more than one solution group, the FloPy plug-in is associated with a specific solution group based on what model it is a part of and the FloPy plug-in callback occurrs only when that solution group is being solved.  To override this behavior and receiving a callback when all solution groups are beings solved, set the run_for_all_solution_groups attribute to True.

### iteration_start(self, sp, ts, iter_num, sln_group)
This method is called for each MODFLOW-6 outer iteration, immediately before the MODFLOW-6 solve method is called.  The iteration_start method has been partially filled out with code that gets the MODFLOW-6 variables (nodelist, hcof, rhs, and bound) from the generic MODFLOW-6 API package you will use.  The code also\loops through each of the stress period records in the current stress period, getting the user-defined values in the record for you.  In this loop you may want to add code that modifies the MODFLOW-6 API package's nodelist, hcof, rhs, and/or bound variables.  Code is already added at the end of this method to commit the modified variables back to MODFLOW-6. 

This method receives the current stress period (sp), the current time step (ts), current iteration number (iter_num) and the modflowapi interface to the solution group currently being solved for (sln_group).  For simulations with more than one solution group, the FloPy plug-in is associated with a specific solution group based on what model it is a part of and the FloPy plug-in callback occurrs only when that solution group is being solved.  To override this behavior and receiving a callback when all solution groups are beings solved, set the run_for_all_solution_groups attribute to True.

### other methods you can override
Additionally you can override the following methods.

* receive_vars(self, simulation, model, package, user_kwargs) - When this method is called your FloPy plug-in receives FloPy objects related to your plugin (simulation, model, package) and also any kwargs passed by the user in the sim.run_simulation() call (user_kwargs).  If you override this method, the receive_vars base class method MUST be called first in order to FloPy plug-ins to function correctly.

* receive_bmi(self, mf6_sim) - This method is called immediately after the MODFLOW-6 API is initialized and is passed a modflowapi Simulation object providing access to the MODFLOW-6 API (mf6_sim).  If you override this method, the recieve_bmi base class method MUST be called first in order to FloPy plug-ins to function correctly.

* iteration_end(self, sp, ts, iter_num, sln_group) - This method is called immediately after the end of the current MODFLOW-6 outer iteration.  This method's parameters are the current stress period (sp), the current time step (ts), current iteration number (iter_num) and the modflowapi interface to the solution group currently being solved for (sln_group).  If you override this method, the iteration_end base class method MUST be called first in order to FloPy plug-ins to function correctly.

* time_step_end(self, sp, ts, converged, sln_group) - This method is called immediately after the end of the current MODFLOW-6 time step.  This method's parameters are the current stress period (sp), the current time step (ts), whether the time step converged on a solution, and the modflowapi interface to the solution group currently being solved for (sln_group).  If you override this method, the time_step_end base class method MUST be called first in order to FloPy plug-ins to function correctly.

* stress_period_end(self, sp, sln_group) - This method is called immediately after the end of the last MODFLOW-6 time step in a stress period.  This method parameters are the current stress period (sp) and the modflowapi interface to the solution group currently being solved for (sln_group).  If you override this method, the stress_period_end base class method MUST be called first in order to FloPy plug-ins to function correctly.

* sim_complete() - This method is called immediately after the MODFLOW-6 simulation is complete.


## Other files used by FloPy plugins

The FloPy plugin template also creates a "gwf-fp_npc.dfn" file in the script's working path that defines the plugin's user interface.  It then runs createpackages.py to create the interface file, "mfgwffp_npc.py".  You can manually edit the dfn file to add more options, stress period vars, or other data blocks to your plugin.  After modifying the dfn you will need to run the "createpackages" method in createpackages.py to update your mfgwffp_npc.py file and add the appropriate code to flopy_npc_package.py to retreive and work with the new data.

Additionally, the FloPy plugin template creates a configuration file, conffpy.py, located in the script’s working path.  The conffpy.py file contains a “flopy_plugins” variable that defines the path to the plugin code file and the name of the plugin’s class. FloPy uses this file to autodetect plugins that are not otherwise installed on your system.

## Example modification to flopy_npc_plugin.py

Find the generated flopy_npc_plugin.py file.  This file will be in your python working directory, the same folder as this notebook is in..  Add the following lines to flopy_npc_plugin.py after the "# ADD YOUR CODE HERE..." comment.  Make sure the indent of the beginning of the code added lines up with the comment.

                rhs_list[row_num] = -q
                hcof_list[row_num] = 0
                cur_nodelist[row_num] = self.mf6_model.usertonode[self.get_node(cellid)] + 1
                bound[row_num, 0] = q
                
This code will use an instance of the API package to apply the amount "q" to the right hand side of the flow equations.

### Using your new plugin

We will now create and run a model to test the new FloPy plugin.  First, a simple simulation with a GHB stress package is created in flopy.

In [ ]:
import numpy as np
from flopy.mf6.modflow.mfgwf import ModflowGwf
from flopy.mf6.modflow.mfgwfdis import ModflowGwfdis
from flopy.mf6.modflow.mfgwfghb import ModflowGwfghb
from flopy.mf6.modflow.mfgwfic import ModflowGwfic
from flopy.mf6.modflow.mfgwfnpf import ModflowGwfnpf
from flopy.mf6.modflow.mfgwfoc import ModflowGwfoc
from flopy.mf6.modflow.mfgwfwel import ModflowGwfwel
from flopy.mf6.modflow.mfims import ModflowIms
from flopy.mf6.modflow.mfgwfsto import ModflowGwfsto
from flopy.mf6.modflow.mfsimulation import MFSimulation
from flopy.mf6.modflow.mftdis import ModflowTdis

sim = MFSimulation(
    sim_name="flopy_npc_test",
    version="mf6",
    exe_name="mf6.exe",
    sim_ws="flopy_npc_test",
    continue_=True,
    memory_print_option="summary",
)

tdis_rc = [(1000.0, 10, 1.0), (500.0, 10, 1.0)]
tdis_package = ModflowTdis(
    sim, time_units="DAYS", nper=2, perioddata=tdis_rc,
    start_date_time="1/1/2010",
)

ims_package = ModflowIms(
    sim,
    filename="flopy_npc_test.ims",
    print_option="ALL",
    complexity="SIMPLE",
    outer_dvclose=0.00001,
    outer_maximum=50,
    under_relaxation="NONE",
    inner_maximum=30,
    inner_dvclose=0.00001,
    linear_acceleration="CG",
    preconditioner_levels=7,
    preconditioner_drop_tolerance=0.01,
    number_orthogonalizations=2,
)

model = ModflowGwf(
    sim, modelname="flopy_npc_test", model_nam_file="flopy_npc_test.nam"
)

nlay = 2
nrow = 10
ncol = 10
dis_package = ModflowGwfdis(
    model,
    length_units="FEET",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=500.0,
    delc=500.0,
    top=100.0,
    botm=[50.0, 0.0],
    pname="mydispkg",
)

ic_package = ModflowGwfic(
    model, strt=80.0
)

oc_package = ModflowGwfoc(
    model,
    head_filerecord=["flopy_npc_test.hds"],
    budget_filerecord=["flopy_npc_test.cbc"],
    headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    printrecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

k = np.empty((nlay,nrow,ncol), float)
for lay in range(0, nlay):
    for row in range(0, nrow):
        for col in range(0, ncol):
            k[lay,row,col] = 120.0 - row * col
            
npf_package = ModflowGwfnpf(
    model,
    save_flows=True,
    alternative_cell_averaging="logarithmic",
    icelltype=1,
    k=k,
    k33=0.01,
)

sto_package = ModflowGwfsto(
    model, save_flows=True, iconvert=1, ss=0.000001, sy=0.15
)

ghb_package = ModflowGwfghb(
    model,
    print_input=True,
    print_flows=True,
    maxbound=10,
    stress_period_data=[((0, 0, 0), 100.0, 60.0), ((0, 0, 1), 100.0, 60.0), ((0, 0, 2), 100.0, 60.0),
                        ((0, 0, 3), 100.0, 60.0), ((0, 0, 4), 100.0, 60.0), ((0, 0, 5), 100.0, 60.0),
                        ((0, 0, 6), 100.0, 60.0), ((0, 0, 7), 100.0, 60.0), ((0, 0, 8), 100.0, 60.0),
                        ((0, 0, 9), 100.0, 60.0),],
)

Next, the FloPy plugin we just created is added to the simulation.

In [ ]:
from mfgwffp_npc import ModflowGwffp_Npc

# Create flopy "npc" plugin stress period data
npc_period = {0: [((0, 4, 4), -2000.0, 1, 2, "test"),
                  ((0, 4, 5), -2500.0, 3, 4, "test2")],
              1: [((0, 4, 4), -500.0, 5, 6, "test3"),
                  ((0, 4, 5), -200.0, 7, 8, "test4")]}
# Create flopy "npc" plugin
npc_plugin = ModflowGwffp_Npc(
    model,
    auxiliary=[("num_1", "num_3")],
    boundnames=True,
    print_input=True,
    print_flows=True,
    stress_period_data=npc_period,
)

Finally, save and run the simulation.

In [ ]:
sim.write_simulation()
sim.run_simulation()

FloPy plugins use the generic "API" package.  Therefore you will see the FloPy plugin's budget in the listing file as the package named "api_npc" which is of type "API" package. Open the "flopy_npc_test.lst" file in the "flopy_npc_test" folder (created next to this notebook) and note that there is a "API_NPC_0" package included in the budget with a rate of 800 ft3/day for stress period 1.

The effects of water withdrawl from the api_npc package can be seen by plotting the heads.  Lower heads can be see in the middle of the model where the api_npc package is withdrawing water.

In [ ]:
heads = model.output.head()
heads.plot(kstpkper=(9, 0), mflay=0, colorbar=True)